In [1]:
import pandas as pd 
import numpy as np 

#Dichiaro il path della cartella del dataset
import os
dataset_folder = "/home/aliquodfahriam/tinyML/CMAPSS/CMAPSS"

In [2]:
#Genero i nomi per le colonne dei sensori
sensors = []
for i in range(1,23): #Nel readme dice che i sensori dovrebbero essere 26, tuttavia dai dati ne emergono soltanto 21, il 22esimo dovrebbe essere pieno di NaN,
                    #Tuttavia mettendo il range da 1 a 22 si crea un warning per cui ci sarebbe una perdita di informazione che non sarebbe desiderabile.  
    sensor = "sensor"+str(i)
    sensors.append(sensor)

print(sensors)

del i, sensor

['sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19', 'sensor20', 'sensor21', 'sensor22']


In [3]:
train = []
test = []
names_train = ['train1', 'train2', 'train3', 'train4']
names_test = ['test1', 'test2', 'test3', 'test4']

files = os.listdir(dataset_folder)
train_files = [file for file in files if file.startswith('train')]
test_files = [file for file in files if file.startswith('test')]
# print(train_files)
# print(test_files)
train_files.sort()
test_files.sort()
print(train_files)
print(test_files)

colnames = ["unit_number", "time_cycle", "op_setting1", "op_setting2", "op_setting3"]

for sensor in sensors:
    colnames.append(sensor)


for file in train_files: 
    df = pd.read_csv(dataset_folder+"/"+file, delimiter=' ', index_col=False, names=colnames)
    train.append(df)

df = train[0]

print(colnames)

#df = pd.read_csv(dataset_folder+"/"+train_files[3], delimiter=' ', index_col=False, names=colnames)


['train_FD001.txt', 'train_FD002.txt', 'train_FD003.txt', 'train_FD004.txt']
['test_FD001.txt', 'test_FD002.txt', 'test_FD003.txt', 'test_FD004.txt']
['unit_number', 'time_cycle', 'op_setting1', 'op_setting2', 'op_setting3', 'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17', 'sensor18', 'sensor19', 'sensor20', 'sensor21', 'sensor22']


In [4]:
## Proviamo a rimuovere gli NaN dai vari dataframe per capire se c'è qualche valore fuori posto in qualche riga.

for df in train: 
    df = df.fillna(value=0, inplace=True)

df = train[0]

counter = 0
for df in train:
    counter += 1 
    value = (df['sensor22'] != 0).any()
    if value: 
        print("Ci sono valori diversi da zero nel dataframe di posto "+ str(counter))
    else:
        print("Tutti i valori sono zero nel dataframe di posto "+str(counter))

#Comprendo da questo che posso procedere a rimuovere la colonna sensor22 da tutti i dataframe in train: 
for df in train: 
    df.drop('sensor22', axis=1, inplace=True)

df = train[0]

Tutti i valori sono zero nel dataframe di posto 1
Tutti i valori sono zero nel dataframe di posto 2
Tutti i valori sono zero nel dataframe di posto 3
Tutti i valori sono zero nel dataframe di posto 4


###### Sensori con valori costanti
Nel papper in esame viene fatto notare come alcuni sensori riportino valori costanti e quindi siano inutili ai fini del nostro esame. Procediamo a verificare che ciò è vero e capiamo se è corretto eliminarli o meno

I sensori da controllare sono: 
- Numero 1
- Numero 5 
- Numero 6
- Numero 10 
- Numero 16 
- Numero 18 
- Numero 19 
- Numero 21 


Da una prima osservazione si nota come non sia possibile tracciare un modello comune per tutti i dataset di training e si debba quindi ricorrere a una preparazione dei dati più *raffinata* che tenga conto del numero di valori per ogni colonna contenente valori appartenenti ad ogni sensore. 

In [5]:
# df = train[0]
# df1 = train[1]
# df2 = train[2]
# df3 = train[3]

In [10]:
unique_values_list = []
unique_values_per_df = []
for dataframe in train: 
    for sensor in sensors: 
        unique_values_frame = dataframe[sensor].unique()
        unique_values_per_df.append(unique_values_frame)
    
    unique_values_list.append(unique_values_per_df)

La variabile *unique_values_list* è composta da 84 colonne poiché la variabile *unique_values_per_df* è composta da 21 colonne, le quali sono proprio quelle dei sensori. Ogni 21 colonne quindi si cambia dataframe.

Operando in modulo 21 possiamo dividere la lista in 3 liste separate per ricavare in maniera più ordinata i dati dei valori unici per colonna. 
Saremo così in grado di comprendere se è possibile eliminare gli stessi sensori per ogni dataframe o se ognuno di essi ha bisogno di particolare attenzione.

Il passo successivo sarà la normalizzazione dei dataframe tramite un operatore di MinMax Normalization